In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
#default_exp classify_image

In [46]:
#export
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *

In [47]:
collection_name = 'food'

In [48]:
#export
foods = read_sql('foods')
# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')

In [49]:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return df.reset_index()

In [50]:
url = 'https://d18mqtxkrsjgmh.cloudfront.net/public/2021-03/Eating%20More%20Ultraprocessed%20%E2%80%98Junk%E2%80%99%20Food%20Linked%20to%20Higher%20CVD%20Risk.jpeg'

In [51]:
search_image(url)

,id,description,category,energy,protein,carb,fat,clip,score
0,1104592,Industrial oil as ingredient in food,Not included in a food category,892.0,0.0,0.0,100.0,"[-0.009690343, -0.0032188592, 0.005953133, 0.0...",0.243913


In [52]:
import numpy as np
import torch
from torch.nn import CosineSimilarity
cos = CosineSimilarity(dim=1, eps=1e-08)

In [53]:
url = 'https://image.shutterstock.com/image-photo/complex-russian-business-lunch-borsch-260nw-1834927213.jpg'
df = search_image(url,head=70)


In [54]:
df

,id,description,category,energy,protein,carb,fat,clip,score
0,1103698,Borscht,Soups,38.0,1.19,4.67,1.81,"[0.04121966, -0.008646833, 0.055504933, -0.009...",0.257814
1,1103322,"Tomato beef soup, prepared with water",Soups,56.0,1.78,8.44,1.71,"[0.034509003, 0.004868654, 0.024858626, -0.008...",0.250869
2,1100351,"Egg white, omelet, scrambled, or fried, with c...",Eggs and omelets,134.0,11.66,1.41,8.87,"[0.04043792, 0.046071332, -0.0073395316, 0.027...",0.248556
3,1103318,"Tomato soup, prepared with water, or ready-to-...",Soups,33.0,0.74,7.72,0.22,"[0.02888626, 0.022543626, 0.027062107, -0.0058...",0.248441
4,1103328,"Tomato vegetable soup, prepared with water",Soups,33.0,0.74,7.72,0.22,"[0.008998611, 0.013765581, 0.02837918, 0.01204...",0.247830
...,...,...,...,...,...,...,...,...,...
65,1099735,"Fish and vegetables including carrots, broccol...",Seafood mixed dishes,105.0,6.28,4.39,7.26,"[0.024300884, 0.039054584, -0.0037791994, 0.00...",0.219463
66,1098069,"Cheese, with wine",Cheese,307.0,16.09,8.85,23.06,"[0.01020844, 0.010757883, 0.031871658, 0.00406...",0.219101
67,1100116,"Chicken or turkey broth, with tomato, home recipe",Soups,49.0,0.85,2.37,4.28,"[0.033205386, 0.04417392, 0.027445301, -0.0299...",0.219018
68,1103705,Gazpacho,Soups,39.0,0.83,4.13,2.38,"[0.027890077, 0.012281019, 0.043598436, -0.014...",0.218835


In [55]:
clip = torch.tensor([np.array(c) for c in df['clip'].values])


In [56]:

best = df.sort_values("score",ascending=False).iloc[0].name
df['score1'] =cos(clip, clip[best])
df['score2'] =cos(clip, clip[df.sort_values("score1",ascending=False).iloc[-1].name])
for i in range(3,10):
    df[f'score{i}'] =cos(clip, clip[df[df.columns[df.columns.str.contains('score')]]         .sum(1).sort_values().index[0]])
df[(df[df.columns[df.columns.str.contains('score')]]==1).any(1)]['description'].drop_duplicates()

0                                               Borscht
18       Crab soup, NS as to tomato-base or cream style
33                                     Wine, table, red
38    Egg omelet or scrambled egg, with meat, tomato...
44    Beef, dumplings, and vegetables including carr...
45    Mushroom soup, with meat broth, prepared with ...
58      Egg casserole with bread, cheese, milk and meat
59    Chicken or turkey garden salad with bacon and ...
Name: description, dtype: object

In [57]:
df[(df[df.columns[df.columns.str.contains('score')]]==1).any(1)]

,id,description,category,energy,protein,carb,fat,clip,score,score1,score2,score3,score4,score5,score6,score7,score8,score9
0,1103698,Borscht,Soups,38.0,1.19,4.67,1.81,"[0.04121966, -0.008646833, 0.055504933, -0.009...",0.257814,1.000000,0.417217,0.539495,0.563640,0.525223,0.525472,0.539677,0.539495,0.586327
18,1100147,"Crab soup, NS as to tomato-base or cream style",Soups,51.0,5.03,1.70,2.55,"[0.037043374, 0.026740106, 0.036702108, -0.062...",0.231759,0.563640,0.402743,0.437821,1.000000,0.412469,0.473144,0.489765,0.437821,0.597414
33,1104473,"Wine, table, red",Wine,85.0,0.07,2.61,0.00,"[0.0046490747, 0.016896492, 0.04259344, -0.091...",0.225351,0.539495,0.313789,1.000000,0.437821,0.430753,0.366805,0.436035,1.000000,0.431080
38,1100294,"Egg omelet or scrambled egg, with meat, tomato...",Eggs and omelets,173.0,11.59,1.33,13.27,"[0.10359659, 0.024291473, 0.036331262, 0.01148...",0.224586,0.417217,1.000000,0.313789,0.402743,0.544955,0.617601,0.514240,0.313789,0.432687
44,1099470,"Beef, dumplings, and vegetables including carr...",Meat mixed dishes,127.0,6.09,11.59,6.29,"[0.033132702, 0.011968696, 0.012818453, 0.0178...",0.223274,0.525223,0.544955,0.430753,0.412469,1.000000,0.472484,0.542670,0.430753,0.529457
45,1103710,"Mushroom soup, with meat broth, prepared with ...",Soups,34.0,1.27,3.76,1.63,"[0.029293507, 0.006133824, 0.036145486, 0.0007...",0.223143,0.586327,0.432687,0.431080,0.597414,0.529457,0.552978,0.440527,0.431080,1.000000
58,1100223,"Egg casserole with bread, cheese, milk and meat",Eggs and omelets,175.0,11.47,10.18,9.60,"[0.010437325, 0.010448935, 0.04977362, 0.03694...",0.221040,0.525472,0.617601,0.366805,0.473144,0.472484,1.000000,0.542830,0.366805,0.552978
59,1099684,Chicken or turkey garden salad with bacon and ...,Poultry mixed dishes,104.0,11.08,2.78,5.61,"[0.04658333, 0.07564571, 0.012737769, -0.00296...",0.220975,0.539677,0.514240,0.436035,0.489765,0.542670,0.542830,1.000000,0.436035,0.440527


In [107]:
!nbdev_build_lib

Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_classfy_image.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted inference.ipynb.
Converted stats.ipynb.
Converted usda_parsing.ipynb.
